# Trabajo Práctico 2
## Tecnología Digital IV
### Integrantes: Marsili, Sanson y Rotmistrovsky

Importamos las librerías que utilizaremos a lo largo de todo el código:

In [17]:
import pandas as pd
import gc
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
from xgboost import XGBClassifier
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample

Generamos el conjunto de validación como una submuestra al azar de los 7 conjuntos de datos y generamos 2 nuevos conjuntos uno para Entrenamiento donde estarán los datos que no han sido utilizados 

In [19]:
# Inicializar listas para almacenar los datos de entrenamiento y validación
train_data_list = []
validation_data_list = []

# Número de observaciones por submuestra
n_samples = 1_000_000 // 7

# Iterar sobre los archivos de datos
for i in range(15, 22):
    # Cargar los datos
    data = pd.read_csv(f"ctr_{i}.csv")
    
    # Verificar que la columna 'Label' existe
    if 'Label' not in data.columns:
        raise KeyError(f"La columna 'Label' no se encuentra en el archivo ctr_{i}.csv")
    
    # Extraer una submuestra aleatoria de 142,857 observaciones para el conjunto de validación
    validation_sample = data.sample(n=n_samples, random_state=42)
    validation_data_list.append(validation_sample)
    
    # Utilizar las observaciones restantes para el conjunto de entrenamiento
    train_sample = data.drop(validation_sample.index)
    train_data_list.append(train_sample)
    
    # Liberar memoria
    del data, validation_sample, train_sample
    gc.collect()

# Combinar las submuestras para formar el conjunto de validación
validation_data = pd.concat(validation_data_list, ignore_index=True)

# Combinar las observaciones restantes para formar el conjunto de entrenamiento
train_data = pd.concat(train_data_list, ignore_index=True)

# Liberar memoria
del validation_data_list, train_data_list
gc.collect()

# Guardar los nuevos conjuntos de datos en archivos CSV
train_data.to_csv("new_train_data.csv", index=False)
validation_data.to_csv("new_validation_data.csv", index=False)

# Balancear el conjunto de entrenamiento
train_class_0 = train_data[train_data['Label'] == 0]
train_class_1 = train_data[train_data['Label'] == 1]

train_class_0_under = resample(train_class_0, replace=False, n_samples=100000, random_state=42)
train_class_1_over = resample(train_class_1, replace=True, n_samples=100000, random_state=42)

train_balanced = pd.concat([train_class_0_under, train_class_1_over])
train_balanced = train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
train_balanced.to_csv('train_balanced_new.csv', index=False)

# Balancear el conjunto de validación
validation_class_0 = validation_data[validation_data['Label'] == 0]
validation_class_1 = validation_data[validation_data['Label'] == 1]

validation_class_0_under = resample(validation_class_0, replace=False, n_samples=100000, random_state=42)
validation_class_1_over = resample(validation_class_1, replace=True, n_samples=100000, random_state=42)

validation_balanced = pd.concat([validation_class_0_under, validation_class_1_over])
validation_balanced = validation_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
validation_balanced.to_csv('validation_balanced_new.csv', index=False)

# Cargar el conjunto de evaluación (sin balanceo)
eval_data = pd.read_csv("ctr_test.csv")

# Guardar el conjunto de evaluación tal cual
eval_data.to_csv('eval_balanced.csv', index=False)


In [ ]:
# Cargar datos de entrenamiento y validación balanceados
train_data = pd.read_csv("train_balanced_new.csv")
validation_data = pd.read_csv("validation_balanced_new.csv")

# Crear variables adicionales
# Interacciones entre niveles de negocio
train_data['action_categorical_0_1'] = train_data['action_categorical_0'].astype(str) + '_' + train_data['action_categorical_1'].astype(str)
validation_data['action_categorical_0_1'] = validation_data['action_categorical_0'].astype(str) + '_' + validation_data['action_categorical_1'].astype(str)

# Transformaciones temporales
train_data['auction_hour'] = pd.to_datetime(train_data['auction_time'], unit='s').dt.hour
validation_data['auction_hour'] = pd.to_datetime(validation_data['auction_time'], unit='s').dt.hour

# Interacciones entre variables numéricas
train_data['bidfloor_age_ratio'] = train_data['auction_bidfloor'] / (train_data['auction_age'] + 1)
validation_data['bidfloor_age_ratio'] = validation_data['auction_bidfloor'] / (validation_data['auction_age'] + 1)

# Variables booleanas combinadas
train_data['auction_boolean_sum'] = train_data[['auction_boolean_0', 'auction_boolean_1', 'auction_boolean_2']].sum(axis=1)
validation_data['auction_boolean_sum'] = validation_data[['auction_boolean_0', 'auction_boolean_1', 'auction_boolean_2']].sum(axis=1)

# Características del dispositivo
device_type_counts = train_data.groupby('device_id')['device_id_type'].nunique().reset_index()
device_type_counts.columns = ['device_id', 'device_type_count']
train_data = train_data.merge(device_type_counts, on='device_id', how='left')
validation_data = validation_data.merge(device_type_counts, on='device_id', how='left')

# Análisis Exploratorio de Datos (EDA) - Conjunto de Entrenamiento
# Figura 1: Distribución de la variable objetivo en el conjunto de entrenamiento
plt.figure(figsize=(10, 6))
sns.countplot(x=train_data["Label"])
plt.title('Distribución de la Variable Objetivo en el Conjunto de Entrenamiento')
plt.xlabel('Label')
plt.ylabel('Frecuencia')
plt.show()

# Figura 2: Mapa de calor de correlación de características numéricas en el conjunto de entrenamiento
plt.figure(figsize=(12, 10))
numeric_features_train = train_data.select_dtypes(include='number').columns
correlation_matrix_train = train_data[numeric_features_train].corr()
sns.heatmap(correlation_matrix_train, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Mapa de Calor de Correlación de Características Numéricas en el Conjunto de Entrenamiento')
plt.show()

# Análisis Exploratorio de Datos (EDA) - Conjunto de Validación
# Figura 3: Distribución de la variable objetivo en el conjunto de validación
plt.figure(figsize=(10, 6))
sns.countplot(x=validation_data["Label"])
plt.title('Distribución de la Variable Objetivo en el Conjunto de Validación')
plt.xlabel('Label')
plt.ylabel('Frecuencia')
plt.show()

# Figura 4: Mapa de calor de correlación de características numéricas en el conjunto de validación
plt.figure(figsize=(12, 10))
numeric_features_val = validation_data.select_dtypes(include='number').columns
correlation_matrix_val = validation_data[numeric_features_val].corr()
sns.heatmap(correlation_matrix_val, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Mapa de Calor de Correlación de Características Numéricas en el Conjunto de Validación')
plt.show()

# Separar características y variable objetivo para los datos de entrenamiento y validación
y_train = train_data["Label"]
X_train = train_data.drop(columns=["Label"])

y_val = validation_data["Label"]
X_val = validation_data.drop(columns=["Label"])

# Liberar memoria
del train_data, validation_data
gc.collect()


In [ ]:
# Separar características y variable objetivo para los datos de entrenamiento y validación
y_train = train_data["Label"]
X_train = train_data.drop(columns=["Label"])

y_val = validation_data["Label"]
X_val = validation_data.drop(columns=["Label"])

# Liberar memoria
del train_data, validation_data
gc.collect()

Realizamos un análisis exploratorio de los datos. Como estos ya se encontraban segmentados en los conjuntos correspondientes realizamos uno para Entrenamiento y otro para Validación.

Continuamos con este bloque de código realiza una búsqueda aleatoria de hiperparámetros para un modelo de clasificación XGBoost, con el objetivo de encontrar la mejor combinación de parámetros que maximice la métrica ROC-AUC en un conjunto de validación. 

In [ ]:
# Define columns
numeric_features = X_train.select_dtypes(include='number').columns
categorical_features = X_train.select_dtypes(exclude='number').columns

# Preprocessing for numeric and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numeric_features),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_features)
    ])

# Define a list of parameters to iterate over
param_grid = {
    'n_estimators': np.arange(100, 550, 50),  # Correcto
    'max_depth': np.arange(3, 15),            # Correcto
    'min_child_weight': np.arange(1, 10),     # Correcto
    'subsample': np.arange(0.5, 1.0, 0.1),   # Corregido: límite superior 1.0
    'colsample_bytree': np.arange(0.5, 1.0, 0.1),  # Corregido: límite superior 1.0
    'gamma': [0, 0.1, 0.5, 1],                 # Correcto
    'learning_rate': [0.01, 0.1, 0.2, 0.3]    # Correcto
}

# Store the best ROC-AUC score and model
best_model = None
best_roc_auc = -np.inf

# Iterate over parameter combinations
for i in range(50):
    # Randomly select a combination of parameters
    params = {key: np.random.choice(values) for key, values in param_grid.items()}
    
    # Create and fit the model
    model = XGBClassifier(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        gamma=params['gamma'],
        learning_rate=params['learning_rate'],
        eval_metric='logloss',
        random_state=22
    )
    
    # Apply preprocessing and fit the model
    pipeline = make_pipeline(preprocessor, model)
    pipeline.fit(X_train, y_train)
    
    # Predict on validation set
    y_val_pred_proba = pipeline.predict_proba(X_val)[:, 1]
    
    # Calculate ROC-AUC
    roc_auc_val = roc_auc_score(y_val, y_val_pred_proba)
    print(f"Iteración {i+1}, ROC-AUC: {roc_auc_val}")
    
    # Update the best model if this one is better
    if roc_auc_val > best_roc_auc:
        best_roc_auc = roc_auc_val
        best_model = pipeline
        print(f"Nuevo mejor modelo en la iteración {i+1} con ROC-AUC: {best_roc_auc}")

In [ ]:
# Once the best model is found, evaluate on the test set
eval_data_num = eval_data.select_dtypes(include='number').drop(columns=["id"])
eval_data_cat = eval_data.select_dtypes(exclude='number')
eval_data_preprocessed = pd.concat([eval_data_num, eval_data_cat], axis=1)

y_preds = best_model.predict_proba(eval_data_preprocessed)[:, 1]

# Create submission file
submission_ctr_21 = pd.DataFrame({"id": eval_data["id"], "Label": y_preds})
submission_ctr_21["id"] = submission_ctr_21["id"].astype(int)
submission_ctr_21.to_csv("xgboost_casero.csv", sep=",", index=False)